In [334]:
Sample = True
One_Hot = True

#### Func

In [312]:
def cat_cleaner(df,col):
    """
    Takes in the df and the column containing messy, sub cats.
    
    pd.get_dummies:
       col                 A | A,B | B | B,A
    0|  A              0|  1    0    0    0
    1| A,B  -onehot->  1|  0    1    0    0
    2|  B              2|  0    0    1    0
    3| B,A             3|  0    0    0    1

    this:
    
       col                       A | B
    0|  A                    0|  1   0
    1| A,B  -cat_cleaner ->  1|  1   1
    2|  B                    2|  0   1
    3| B,A                   3|  1   1
    
    
    
    """
    def cat(text):
        text =[word for word in text.split(',')]
        text =[word.strip() for word in text]
        return(text)
    
    
    df[col] = df[col].astype('str')
    clean_cats = df[col].apply(lambda x:cat(x))
    p = pd.get_dummies(clean_cats.apply(pd.Series).stack()).sum(level=0).reset_index(drop=True)
    return p

def text_cleaner(text):
    """
    clean_str = text_cleaner(dirty_string)
    """
    
    text = text.lower()
    text = text.replace('<!--DONOTREMOVEESSAYDIVIDER-->',' ')
    text = text.replace('\n' , ' ')
    return(text)

def plot_empties(bad_form):
    plt.figure(figsize=(15,15))

    i = 1
    for j in tqdm_notebook(bad_form):
        plt.subplot(230+i)
        sns.heatmap(pd.DataFrame(bad_form[j].isnull().sum()/bad_form[j].shape[0]*100),
                    annot=True,cmap=sns.color_palette("cool"),linewidth=1,linecolor="white")
        plt.title(j)
        i+=1

    plt.subplots_adjust(wspace = 1.6)
    return

def compress(df,encode=[],num=[],byte_str = False):
    pre_ = df[num+encode].memory_usage(deep=True)

    for i in num:
        if df[i].astype(np.float16).memory_usage(deep=True)<df[i].memory_usage(deep=True):
            df[i] = df[i].astype(np.float16)
        else:
            pass

    if byte_str:
        for j in encode:
            if df[j].astype(np.string_).memory_usage(deep=True)<df[j].memory_usage(deep=True):
                df[j] = df[j].astype(np.string_)
            else:
                pass
    else:
        for j in encode:
            if df[j].astype(str).memory_usage(deep=True)<df[j].memory_usage(deep=True):
                df[j] = df[j].astype(str)
            else:
                pass

            
            
    post_ = df[num+encode].memory_usage(deep=True)
    print("Data Usage - change:{}".format((post_-pre_)))
    print("Total Change: {}".format((post_-pre_).sum()))
    return

### Imports

In [313]:
import gc
import os
import calendar
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt

In [314]:
os.listdir('./Input/New')

['.DS_Store',
 'Donations.csv',
 'Donors.csv',
 'Projects.csv',
 'Resources.csv',
 'Schools.csv',
 'Teachers.csv']

### Data Aggregation

In [5]:
bad_form = {#'Donations' : pd.read_csv('./Input/New/Donations.csv'),
            #'Donors'    : pd.read_csv('./Input/New/Donors.csv'),
            
            'Projects'  : pd.read_csv('./Input/New/Projects.csv'),
            'Resources' : pd.read_csv('./Input/New/Resources.csv'),
            'Schools'   : pd.read_csv('./Input/New/Schools.csv'),
            'Teachers'  : pd.read_csv('./Input/New/Teachers.csv')}

In [335]:
#How much of the data do you want to use?
if Sample: df = bad_form['Projects'][:1000].copy(deep=True)
else: df = bad_form['Projects'].copy(deep=True)

In [336]:
#Projects

#Structure Target
df['Project Current Status'] = df['Project Current Status'].apply(lambda x: 1 if x == 'Fully Funded' else 0)

#Adding dt features
df['Project Posted Date'] = pd.to_datetime(df['Project Posted Date'])
df['Project Posted Year'] = df['Project Posted Date'].dt.year.astype(str)
df['Project Posted Month'] = df['Project Posted Date'].dt.month
df['Project Posted Month'] = df['Project Posted Month'].apply(lambda x: calendar.month_abbr[x])

#cols
encode_cols = ['Project Type','Project Posted Year','Project Posted Month',
               'Project Grade Level Category','Project Resource Category',]
num_cols    = ['Project Cost']

#del
for i in ['Teacher Project Posted Sequence', 'Project Fully Funded Date', 'Project Expiration Date', 'Project Subject Subcategory Tree', 'Project Posted Date']:
    del df[i]
gc.collect()

22304

In [337]:
#Null values from all data sources

#plot_empties(bad_form)

In [338]:
#Teachers

#Treating Nulls in 'Teachers'
bad_form['Teachers']['Teacher Prefix'] = bad_form['Teachers']['Teacher Prefix'].apply(
    lambda x: 'Teacher' if x in ['Mx.', np.nan] else x)

#merging teacher to df
df = df.merge(bad_form['Teachers'],on='Teacher ID').copy(deep=True)

#cols
encode_cols += ['Teacher Prefix']
num_cols    += []

#del
for i in ['Teacher ID','Teacher First Project Posted Date']:
    del df[i]
gc.collect()

57

In [339]:
#School

#very few, dropping nulls
bad_form['Schools'] = bad_form['Schools'].dropna()

#merging school to df
df = df.merge(bad_form['Schools'],on='School ID')
df['School Zip'] = df['School Zip'].astype(str)

#cols
encode_cols += ['School Metro Type','School State','School District','School Zip']
num_cols    += ['School Percentage Free Lunch']

#del
for i in ['School Name','School ID','School City','School County']:
    del df[i]
gc.collect()

64

In [340]:
# Resource Management

#more funding features
bad_form['Resources']['Resource Total Cost'] = bad_form['Resources']['Resource Quantity'] * bad_form['Resources']['Resource Unit Price']
funding_agg = bad_form['Resources'].groupby('Project ID').agg(
    {'Resource Item Name': ['nunique'], 
     'Resource Quantity': ['sum'], 
     'Resource Total Cost': ['mean', 'sum','median','max','min']})

funding_agg.columns = ['Project num Unique Resources',
                   'Total Resource Quantity', 
                   'Mean Resource Cost', 
                   'Total Project Cost',
                   'Median Resource Cost',
                   'Most exp Resource Cost',
                   'Least exp Resource Cost']

#cols
encode_cols += []
num_cols    += list(funding_agg.columns.values)

#merging funding to df
funding_agg.reset_index(inplace=True)
df = df.merge(funding_agg,on='Project ID')

del funding_agg
gc.collect()

14

In [341]:
del df['Project ID']

In [367]:
df = df.dropna()

In [342]:
s = set(encode_cols+num_cols)
df[[x for x in df.columns if x not in s]].head(5)

,Project Title,Project Essay,Project Short Description,Project Need Statement,Project Subject Category Tree,Project Current Status
0,Stand Up to Bullying: Together We Can!,Did you know that 1-7 students in grades K-12 ...,Did you know that 1-7 students in grades K-12 ...,"My students need 25 copies of ""Bullying in Sch...",Applied Learning,1
1,Learning in Color!,"Help us have a fun, interactive listening cent...","Help us have a fun, interactive listening cent...","My students need a listening center, read alon...","Applied Learning, Literacy & Language",0
2,Help Second Grade ESL Students Develop Languag...,Visiting or moving to a new place can be very ...,Visiting or moving to a new place can be very ...,My students need beginning vocabulary audio ca...,Literacy & Language,1
3,Help Bilingual Students Strengthen Reading Com...,Students at our school are still working hard ...,Students at our school are still working hard ...,My students need one copy of each book in The ...,Literacy & Language,1
4,Help Us Make Each Minute Count!,"""Idle hands"" were something that Issac Watts s...","""Idle hands"" were something that Issac Watts s...","My students need items such as Velcro, two pou...",Special Needs,1


In [343]:
#compressing data
compress(df,encode_cols,num_cols)
gc.collect()

Data Usage - change:Index                              0
Project Cost                   -5880
School Percentage Free Lunch   -5880
Project num Unique Resources   -5880
Total Resource Quantity        -5880
Mean Resource Cost             -5880
Total Project Cost             -5880
Median Resource Cost           -5880
Most exp Resource Cost         -5880
Least exp Resource Cost        -5880
Project Type                       0
Project Posted Year                0
Project Posted Month               0
Project Grade Level Category       0
Project Resource Category          0
Teacher Prefix                     0
School Metro Type                  0
School State                       0
School District                    0
School Zip                         0
dtype: int64
Total Change: -52920


35

In [326]:
#del bad_form

### Encoding / Scaling

In [346]:
if One_Hot:
    df.merge(pd.get_dummies(df[encode_cols])
             ,left_index=True,right_index=True)
    for i in encode_cols:
        del df[i]
else:
    for c in tqdm_notebook(encode_cols):
        encod = LabelEncoder()
        encod.fit(df[c].astype(str))
        df[c] = encod.transform(df[c].astype(str))
    del encod

In [347]:
p = cat_cleaner(df,'Project Subject Category Tree')
df = df.merge(p,left_index=True,right_index=True)
del df['Project Subject Category Tree'], p

In [348]:
gc.collect()

8155

### Simple Model

In [350]:
text_cols = ['Project Title','Project Essay','Project Short Description','Project Need Statement']

In [354]:
df.dtypes

Project Title                    object
Project Essay                    object
Project Short Description        object
Project Need Statement           object
Project Cost                    float16
Project Current Status            int64
School Percentage Free Lunch    float16
Project num Unique Resources    float16
Total Resource Quantity         float16
Mean Resource Cost              float16
Total Project Cost              float16
Median Resource Cost            float16
Most exp Resource Cost          float16
Least exp Resource Cost         float16
Applied Learning                  uint8
Health & Sports                   uint8
History & Civics                  uint8
Literacy & Language               uint8
Math & Science                    uint8
Music & The Arts                  uint8
Special Needs                     uint8
dtype: object

In [351]:
text_cols

['Project Title',
 'Project Essay',
 'Project Short Description',
 'Project Need Statement']

In [368]:
X = df.drop(text_cols+['Project Current Status'],axis=1)
y = df['Project Current Status']

In [369]:
from sklearn.model_selection import train_test_split

In [370]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [371]:
from sklearn.linear_model import LogisticRegression

In [372]:
lr = LogisticRegression()

In [373]:
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [374]:
lr.score(X_test,y_test)

/Users/Nahel/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


0.7355371900826446

In [ ]:
from modul

### Text Editing

In [21]:
text_cols = ['Project Title','Project Essay','Project Short Description','Project Need Statement']

In [ ]:
x_words = ['student','students','education']

In [54]:
for i in text_cols:
    df[i] = text_cleaner

In [31]:
df.iloc[1]['Project Title']

'learning in color!'

In [32]:
def text_cleaner(text,extra_words = ['student','students','education']):
    """
    clean_str = text_cleaner(dirty_string)
    """
    text = text.lower()
    text = text.replace('<!--DONOTREMOVEESSAYDIVIDER-->',' ')
    text = text.replace('\n' , ' ')
    
    #drops extra words
    extra_words = [x for x in 'abcdefghijklmnopqrstuvwxyz']
    for j in extra_words:
        extra_words.append(j)
    extra_words += stopwords.words("english")
    text = ' '.join([word for word in text.split() if word not in extra_words])
    

    return(text)

In [52]:
df['Project Title'].apply(lambda x: tex(x))

0                          stand bullying: together can!
1                                        learning color!
2      help second grade esl develop language speak read
3        help bilingual strengthen reading comprehension
4                             help us make minute count!
5                                     time... time kids!
6                                     experiencing india
7                                      come read upfront
8                                  grammar, grammar need
9                 little learners love listen literature
10                             time grab "bully" horns!!
11      1st grade mathematicians need learning materials
12                           want become somebody, read!
13                         "i good pencil eraser today!"
14                                   kindle love reading
15                                         thinking feet
16                                              feelings
17             learning english

Index(['Project ID', 'Teacher Project Posted Sequence', 'Project Type',
       'Project Title', 'Project Essay', 'Project Short Description',
       'Project Need Statement', 'Project Subject Category Tree',
       'Project Subject Subcategory Tree', 'Project Grade Level Category',
       'Project Resource Category', 'Project Cost', 'Project Posted Date',
       'Project Current Status', 'Teacher Prefix', 'School Name',
       'School Metro Type', 'School Percentage Free Lunch', 'School State',
       'School Zip', 'School City', 'School County', 'School District',
       'Project num Unique Resources', 'Total Resource Quantity',
       'Mean Resource Cost', 'Total Project Cost', 'Median Resource Cost',
       'Most exp Resource Cost', 'Least exp Resource Cost'],
      dtype='object')

In [41]:
all_stopwords = [x for x in 'abcdefghijklmnopqrstuvwxyz']
for j in ['student','students','education']:
    all_stopwords.append(j)
all_stopwords += stopwords.words("english")

## Structured Encoding

In [12]:
cat_cols = ['Project Type']

In [13]:
p = cat_cleaner(df,'Project Subject Category Tree')

In [14]:
p.head()

,Applied Learning,Health & Sports,History & Civics,Literacy & Language,Math & Science,Music & The Arts,Special Needs
0,1,0,0,0,0,0,0
1,1,0,0,1,0,0,0
2,0,0,0,1,0,0,0
3,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1
